In [1]:
pip install -U sentence-transformers



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install elasticsearch 


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import elasticsearch
print(elasticsearch.__version__)

(8, 11, 1)


In [4]:
import time
import sys
import json
import random
import os
from pprint import pprint
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer


In [5]:
class Search:
    def __init__(self):
        PORT="https://localhost:9200"
        AUTH=("elastic", "KpLvmggcUaTz52ug=B8-")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.es = Elasticsearch(PORT,basic_auth=AUTH,verify_certs=False)  
        client_info = self.es.info()
        print('Connected to Elasticsearch!')
        pprint(client_info.body)
        
    def get_embedding(self, text):
        return self.model.encode(text)
    
    def insert_document(self, index, document):
        return self.es.index(index=index, document={
            **document,
            'embedding': self.get_embedding(document['Name']),
        })
    
    def insert_documents(self, index, documents):
        for document in documents:
            self.insert_document(index,document)

    def search(self, index, **query_args):
        return self.es.search(index=index, **query_args)
    
    def create_embedding_index(self,index):
        self.es.indices.delete(index=index, ignore_unavailable=True)
        self.es.indices.create(index=index, mappings={
            'properties': {
                'embedding': {
                    'type': 'dense_vector',
                }
            }
        })

In [6]:
es = Search()

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'JM976w_xRBSKJE0gtXKEmg',
 'name': '8e79d9b218a8',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2023-12-08T11:33:53.634979452Z',
             'build_flavor': 'default',
             'build_hash': '64cf052f3b56b1fd4449f5454cb88aca7e739d9a',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.8.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.11.3'}}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
with open('json.json', 'rt') as f:
    documents = json.loads(f.read())

In [8]:
es.create_embedding_index('all_documents')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
es.insert_documents('all_documents',documents)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

In [10]:
query = "KODAK"
es.search('all_documents',
    query={
        'match': {
            'Name': {
                'query': query
            }
        }
    }
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'took': 23, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 8, 'relation': 'eq'}, 'max_score': 4.70459, 'hits': [{'_index': 'all_documents', '_id': 'ef3cao0By_ZU5eabj6wJ', '_score': 4.70459, '_ignored': ['Reviews.keyword'], '_source': {'Name': 'Kodak AZ401RD Point & Shoot Digital Camera with 3" LCD, Red', 'Price': '169.99', 'Details': ['You’ll get well defined, detailed images thanks to the AZ401’s 16 million pixels. That’s ample resolution to crop, zoom, or enlarge your photos however the photographer in you desires.', 'With a 24mm wide angle lens, there’s no sacrificing what you want in the frame. So go ahead and let your camera see what you see - the sky’s the limit!', 'The Optical Image Stabilization will iron out any small movements as you’re focused on getting your shot, so you never have to be afraid to go for what you want.', 'Whether you’re in the moment, or modifying your work at a later t

In [11]:
query = "KODAK"
es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },
        
    )

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'took': 41, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10, 'relation': 'eq'}, 'max_score': 0.77607435, 'hits': [{'_index': 'all_documents', '_id': 'lP3cao0By_ZU5eabk6yH', '_score': 0.77607435, '_ignored': ['Reviews.keyword'], '_source': {'Name': 'KODAK PIXPRO FZ55-BK 16MP Digital Camera 5X Optical Zoom 28mm Wide Angle 1080P Full HD Video 2.7" LCD Vlogging Camera (Black)', 'Price': '119.99', 'Details': ['16 Megapixel CMOS Sensor', '5X Optical Zoom - 28mm Wide Angle Lens', '1080P Full HD Video - Vlogging Camera', 'SD Card Compatibility: At least Class 10, no more than 512GB (SD, SDHC, SDXC)', '2.7" LCD Screen - Rechargeable Li-Ion Battery'], 'Ratings': '4.2', 'Rates': '842', 'Reviews': ["Bought this camera for a simple camera to use while traveling when I didn't want to take my huge Panasonic camera but still wanted a camera that wasn't on my phone, for safety purposes and also the anxiety of pos

In [12]:
def Recommender(index,selected_doc):
    results = es.search(index,
        query={
            'multi_match': {
                'query': selected_doc['Name'],
                'fields': ['Name', 'Details'],
            }
        }   
    )
    return results['hits']['hits']



In [13]:
Recommender('all_documents',documents[2])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'_index': 'all_documents',
  '_id': 'Xf3cao0By_ZU5eabiaw5',
  '_score': 46.06398,
  '_ignored': ['Details.keyword', 'Reviews.keyword', 'Link.keyword'],
  '_source': {'Name': 'Digital Camera, Kids Camera FHD 1080P 44MP Small Digital Camera with 32GB SD Card Vlogging Camera 16X Digital Zoom, Point and Shoot Digital Camera Gift for Kids Boys Girls Teens Students - Black',
   'Price': '36.79',
   'Details': ['[FHD 1080P & 44MP Camera]: This digital camera cheap with 44 MP picture pixels that can shoot 1080p videos, will help you get exciting photos or videos, and our kids digital camera comes with a 32GB SD card and additional features like face detection, smile capture, self timer (3S/5S/10S), 16X digital zoom, fill light, Anti- shake function etc, if you want a small digital cameras to vlog and take cute pics, it is great for memories.',
    "[Portable Travel Digital Camera]: Our compact digital camera is smaller than a smart phone, enough small to fit your pocket or backpack, suitable

In [14]:
selected_docs = random.sample(documents,k = 20)

In [15]:
es.insert_documents('selected_documents',selected_docs)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

In [18]:
adjacency_dictionary = {}
for document in selected_docs:
    adjacency_dictionary[document["Name"]] = Recommender('selected_documents',document)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

In [19]:
vertices = sorted(adjacency_dictionary.keys())
matrix = [[next((item['_score'] for item in adjacency_dictionary[vertex] if item['_source']['Name'] == v), 0) for v in vertices] for vertex in vertices]
print(matrix)


[[40.4737, 18.01145, 0, 0, 0, 0, 17.986586, 14.452533, 11.506586, 15.718798, 0, 5.6499467, 0, 22.51877, 0, 7.440077, 0, 0], [11.124021, 47.200592, 0, 0, 0, 6.922467, 9.164645, 7.693335, 9.080707, 7.8603234, 0, 0, 0, 10.090092, 0, 13.98205, 0, 0], [5.973182, 0, 43.442757, 20.816395, 14.457434, 25.393982, 0, 0, 0, 0, 20.312939, 0, 14.220672, 4.826472, 16.423462, 0, 0, 10.593285], [6.464173, 0, 17.091362, 49.137085, 17.476107, 12.810308, 0, 5.2660537, 0, 0, 19.759327, 0, 27.972113, 0, 22.47083, 0, 0, 14.343843], [0, 7.691093, 17.330517, 21.413263, 53.114468, 21.478495, 0, 0, 0, 5.8957205, 11.4318075, 0, 11.616989, 0, 12.115711, 0, 0, 10.422951], [3.4957159, 0, 25.01294, 13.350293, 14.927908, 43.514446, 0, 0, 0, 0, 13.952416, 0, 13.250407, 4.6087627, 21.19627, 0, 0, 13.200237], [11.382424, 13.91619, 0, 0, 4.5692167, 0, 44.876587, 21.794891, 26.113695, 9.863741, 0, 0, 0, 13.1036, 0, 8.346944, 0, 0], [11.658785, 8.828586, 0, 0, 0, 0, 21.959082, 29.13927, 19.579922, 7.5992227, 0, 0, 3.5483787

In [37]:
query = "Vlogging Camera"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },
        
    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

4K Digital Camera for Photography 48MP Vlogging Camera for YouTube Vloggers Gifts for Beginners Teens with Rechargeable Battery
VJIANGER 2024 Upgrade 4K Digital Camera for Photography Vlogging Camera for YouTube 64MP Photography Camera with WiFi, 52mm Fixed Lens, 4.0" Touch Screen, 32GB SD Card & Camera Bag(W05-BlackS2)
VJIANGER 2024 Upgrade 4K Digital Camera for Photography Vlogging Camera for YouTube 64MP Photography Camera with WiFi, 52mm Fixed Lens, 4.0" Touch Screen, 32GB SD Card & Camera Bag(W05-BlackS2)
VJIANGER 4K Digital Camera 48MP WiFi Vlogging Camera for YouTube with 180° Flip Screen, 16X Digital Zoom, 52mm Wide Angle & Macro Lens, 2 Batteries, 32GB TF Card(W02 Black4)
Digital Camera for Photography VJIANGER 4K 48MP Vlogging Camera for YouTube with WiFi, 180 Degree Flip Screen, 16X Digital Zoom, 52mm Wide Angle & Macro Lens, 2 Batteries, 32GB TF Card(W02 Black6)
NIKICAM Digital Camera for Photography 4K 56MP Vlogging Camera for YouTube with 16X Digital Zoom, 2 Rechargeable 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [36]:
query = "Photography"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },
        
    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

Digital Camera 1080P Kids Camera with 16X Zoom - 44MP Anti Shake Point and Shoot Digital Cameras for Photography | Compact Small Camera for Kids Girls Boys Teens Beginners (SD Card Not Include)
Digital Camera 1080P Kids Camera with 16X Zoom - 44MP Anti Shake Point and Shoot Digital Cameras for Photography | Compact Small Camera for Kids Girls Boys Teens Beginners (SD Card Not Include)
VETEK Digital Cameras for Photography, 4K 48MP Vlogging Camera 16X Digital Zoom Manual Focus Students Compact Camera with 52mm Wide-Angle Lens & Macro Lens, 32G Micro Card and 2 Batteries (Pink)
KODAK PIXPRO AZ405-BK 20MP Digital Camera 40X Optical Zoom 24mm Wide Angle Lens Optical Image Stabilization 1080P Full HD Video 3" LCD Vlogging Camera (Black)
Digital Camera, Auto Focus FHD 4K Vlogging Camera with Dual Camera 48MP 16X Digital Zoom Kids Compact Camera with 32GB Memory Card Portable Point and Shoot Cameras for Teens Beginner Adult,Black
Digital Camera, 48MP HD Autofocus Anti-Shake 18X Digital Zoom V

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [33]:
query = "video camera"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },
        
    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

G-Anica Digital Camera 4K Camcorder 48MP 3.0" IPS Flip Screen Video Camera 16X Digital Zoom Vlogging Camera for YouTube, Compact Camera with 32GB SD Card and Wide Angle Lens Macro Lens…
G-Anica 4K Digital Cameras for Photography，48MP/60FPS Video Camera for Vlogging, WiFi & App Control Vlogging Camera for YouTube, Small Camera with 32GB TF Card.Wide-Angle & Macro Lens
KODAK PIXPRO FZ45-WH 16MP Digital Camera 4X Optical Zoom 27mm Wide Angle 1080P Full HD Video 2.7" LCD Vlogging Camera (White)
KODAK PIXPRO FZ55-BK 16MP Digital Camera 5X Optical Zoom 28mm Wide Angle 1080P Full HD Video 2.7" LCD Vlogging Camera (Black)
Digital Camera for Photography VJIANGER 4K 48MP Vlogging Camera for YouTube with WiFi, 180 Degree Flip Screen, 16X Digital Zoom, 52mm Wide Angle & Macro Lens, 2 Batteries, 32GB TF Card(W02 Black6)
4K Digital Camera for Photography 48MP Vlogging Camera for YouTube Vloggers Gifts for Beginners Teens with Rechargeable Battery
VJIANGER 4K Digital Camera 48MP WiFi Vlogging Camera 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [34]:
query = "Kids Camera"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },
        
    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

Digital Camera 1080P Kids Camera with 16X Zoom - 44MP Anti Shake Point and Shoot Digital Cameras for Photography | Compact Small Camera for Kids Girls Boys Teens Beginners (SD Card Not Include)
Digital Camera 1080P Kids Camera with 16X Zoom - 44MP Anti Shake Point and Shoot Digital Cameras for Photography | Compact Small Camera for Kids Girls Boys Teens Beginners (SD Card Not Include)
Digital Camera, Kids Camera FHD 1080P 44MP Small Digital Camera with 32GB SD Card Vlogging Camera 16X Digital Zoom, Point and Shoot Digital Camera Gift for Kids Boys Girls Teens Students - Black
GREENKINDER Upgrade Kids Camera Christmas Birthday Gifts for Boys Girls Age 3-12, Children Digital Video Cameras with Flash, 2 Inch Screen Selfie Camcorder HD 1080P Video 32GB Black
MEETRYE Instant Print Camera for Kids - Christmas and Birthday Gift Ideas - Digital Toy Camera for Girls and Boys Ages 4 to 14
Instant Print Camera - PROGRACE Kids Camera Toy 1080P HD 0 Ink Print Photo Christmas Birthday Gifts for Girl

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [35]:
query = "Camera 4K"
res = es.search('all_documents',
        knn={
            'field': 'embedding',
            'query_vector': es.get_embedding(query),
            'num_candidates': 50,
            'k': 10,
        },
        
    )
res = res['hits']['hits']
for i in res:
    print(i['_source']['Name'])

4K Digital Camera for Photography Auto-Focus 4K Camera with 180° 3.0 inch Flip Screen 16X Anti-Shake Vlogging Camera for YouTube Video Compact Cameras with SD Card, 2 Batteries and Battery Charger
4K Digital Camera for Photography Auto-Focus 4K Camera with 180° 3.0 inch Flip Screen 16X Anti-Shake Vlogging Camera for YouTube Video Compact Cameras with SD Card, 2 Batteries and Battery Charger
Newest 4K Digital Camera for Photography and Video, 48MP Vlogging Camera with SD Card Autofocus Anti-Shake, 3'' 180° Flip Screen 16X Zoom Digital Camera with Flash, Compact Digital Camera for Travel
4K Digital Camera for Photography Autofocus, Upgraded 48MP Vlogging Camera for YouTube with SD Card, 3" 180 Flip Screen Compact Travel Camera with 16X Digital Zoom, Flash, Anti-Shake, 2 Batteries
YEEIN 4K Cameras for Photography and Video, 3'' Screen Compact Digital Camera with Leather Case, Autofocus 48MP Vlogging Camera with 32G Card, Dual Lens 4k Camera with Built-in 7 Color Filters
4K Digital Cameras

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [40]:
testing_documents = documents[:5]

In [43]:
es.insert_documents('testing_documents',testing_documents)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

In [44]:
adjacency_dictionary = {}
for document in testing_documents:
    adjacency_dictionary[document["Name"]] = Recommender('testing_documents',document)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

In [45]:
vertices = sorted(adjacency_dictionary.keys())
prediction_matrix = [[next((item['_score'] for item in adjacency_dictionary[vertex] if item['_source']['Name'] == v), 0) for v in vertices] for vertex in vertices]
print(prediction_matrix)

[[25.103662, 6.4932923, 5.81129, 4.9018664, 11.566331], [13.581725, 24.47444, 1.9819951, 4.6392694, 7.939229], [2.54001, 1.7531276, 18.232708, 10.275377, 1.8881803], [2.1797688, 1.7531276, 11.129893, 19.924057, 1.8881803], [8.770007, 6.4932923, 0.9791776, 4.84325, 14.817237]]


In [ ]:
ground_truth = [[1,4,3,5,2],
                [2,1,5,4,3],
                [3,4,1,2,5],
                [3,5,2,1,4],
                [2,3,5,4,1]]

In [ ]:
def rank_predictions(prediction_matrix):
    ranked_predictions = []
    for i, row in enumerate(prediction_matrix):
        ranked_predictions.append(row)
        ranked_predictions[i] = sorted(ranked_predictions[i], reverse = True)
        for j in range (len(prediction_matrix)):
            indx = ranked_predictions[i].index(row[j])
            row [j] = indx + 1
    return  prediction_matrix

def apk(actual, predicted, k):
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p == actual[i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k):
    predicted = rank_predictions(predicted)
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
mapk(ground_truth,  prediction_matrix, 5)